In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import normalized_mutual_info_score,adjusted_mutual_info_score
import pandas as pd
import networkx as nx
import numpy as np
import torch
from torch_geometric.data import Data
import torch
import pandas as pd
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GATConv
from torch_geometric.loader import DataLoader
import random
from sklearn.model_selection import train_test_split

In [2]:
#pip install uci-dataset

In [57]:
import uci_dataset as dataset

data = dataset.load_audiology()

In [58]:
data

,age_gt_60,air,airBoneGap,ar_c,ar_u,bone,boneAbnormal,bser,history_buzzing,history_dizziness,...,s_sn_gt_2k,s_sn_gt_4k,speech,static_normal,tymp,viith_nerve_signs,wave_V_delayed,waveform_ItoV_prolonged,indentifier,Class
0,f,mild,f,normal,normal,NaN,t,NaN,f,f,...,f,f,normal,t,a,f,f,f,p1,cochlear_unknown
1,f,moderate,f,normal,normal,NaN,t,NaN,f,f,...,f,f,normal,t,a,f,f,f,p2,cochlear_unknown
2,t,mild,t,NaN,absent,mild,t,NaN,f,f,...,f,f,normal,t,as,f,f,f,p3,mixed_cochlear_age_fixation
3,t,mild,t,NaN,absent,mild,f,NaN,f,f,...,f,f,normal,t,b,f,f,f,p4,mixed_cochlear_age_otitis_media
4,t,mild,f,normal,normal,mild,t,NaN,f,f,...,f,f,good,t,a,f,f,f,p5,cochlear_age
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,t,mild,f,absent,normal,mild,t,NaN,f,f,...,f,f,very_good,t,a,f,f,f,p196,cochlear_age
196,t,mild,f,normal,absent,mild,f,NaN,f,f,...,f,f,very_good,t,c,f,f,f,p197,mixed_cochlear_age_otitis_media
197,f,normal,f,normal,normal,unmeasured,f,degraded,f,f,...,f,f,normal,f,a,f,f,f,p198,possible_brainstem_disorder
198,t,mild,f,normal,normal,unmeasured,f,NaN,f,f,...,f,f,very_good,t,a,f,f,f,p199,cochlear_age


In [4]:
data_g = data[data["age_gt_60"] == "t"]
data_l = data[data["age_gt_60"] == "f"]

In [5]:
data_g=data_g.apply(LabelEncoder().fit_transform).dropna()
data_l=data_l.apply(LabelEncoder().fit_transform).dropna()

In [6]:
features_g = data_g.columns[:-1]
classLabel_g = data_g.columns[-1:][0]
features_l = data_l.columns[:-1]
classLabel_l = data_l.columns[-1:][0]

In [7]:
edge_rows = []
edge_colNames = ["i","j","mi"]
out_rows = []
out_colNames = ["f","mi"]
node_rows = []
node_colNames = ["f","mi"]

In [8]:
for f1 in features_g:
    for f2 in features_g:
        mi = normalized_mutual_info_score(data_g[f1], data_g[f2])
        if f1 == f2:
            continue
        f1g = f1 + "g"
        f2g = f2 + "g"
        t1 = (f1g,f2g,mi)
        t2 = (f2g,f1g,mi)
        edge_rows.append(t1)
        edge_rows.append(t2)
        
    miClass = normalized_mutual_info_score(data_g[f1], data_g[classLabel_g])
    f1g = f1 + "g"
    t_out = (f1g,miClass)
    out_rows.append(t_out)


In [9]:
for f1 in features_l:
    for f2 in features_l:
        if f1 == f2:
            continue
        mi = normalized_mutual_info_score(data_l[f1], data_l[f2])
        f1l = f1 + "l"
        f2l = f2 + "l"
        t1 = (f1l,f2l,mi)
        t2 = (f2l,f1l,mi)
        edge_rows.append(t1)
        edge_rows.append(t2)
        
    miClass = normalized_mutual_info_score(data_l[f1], data_l[classLabel_l])
    f1l = f1 + "l"
    t_out = (f1l,miClass)
    out_rows.append(t_out)

In [10]:
for f1 in features_l:
    mi = 1
    f1l = f1 + "l"
    f1g = f1 + "g"
    t1 = (f1l,f1g,mi)
    t2 = (f1g,f1l,mi)
    edge_rows.append(t1)
    edge_rows.append(t2)

In [11]:
for f in features_g:
    fl = f + "l"
    fg = f + "g"
    rl = (fl,1)
    rg = (fg,1)
    node_rows.append(rl)
    node_rows.append(rg)

In [12]:
len(node_rows)

140

In [13]:
edge_df = pd.DataFrame(edge_rows,
                  columns=edge_colNames)

In [14]:
edge_df

,i,j,mi
0,age_gt_60g,airg,6.422528e-16
1,airg,age_gt_60g,6.422528e-16
2,age_gt_60g,airBoneGapg,0.000000e+00
3,airBoneGapg,age_gt_60g,0.000000e+00
4,age_gt_60g,ar_cg,1.573861e-15
...,...,...,...
19455,wave_V_delayedg,wave_V_delayedl,1.000000e+00
19456,waveform_ItoV_prolongedl,waveform_ItoV_prolongedg,1.000000e+00
19457,waveform_ItoV_prolongedg,waveform_ItoV_prolongedl,1.000000e+00
19458,indentifierl,indentifierg,1.000000e+00


In [15]:
edge_df.to_csv("data/edge.csv")

In [16]:
out_df = pd.DataFrame(out_rows,
                  columns=out_colNames)

In [17]:
out_df.to_csv("data/out.csv")

In [18]:
out_df

,f,mi
0,age_gt_60g,0.000000
1,airg,0.248551
2,airBoneGapg,0.215257
3,ar_cg,0.178933
4,ar_ug,0.213519
...,...,...
135,tympl,0.491261
136,viith_nerve_signsl,0.043010
137,wave_V_delayedl,0.074491
138,waveform_ItoV_prolongedl,0.054318


In [19]:
node_df = pd.DataFrame(node_rows,
                  columns=node_colNames)

In [20]:
node_df.to_csv("data/node.csv")

In [21]:
node_df

,f,mi
0,age_gt_60l,1
1,age_gt_60g,1
2,airl,1
3,airg,1
4,airBoneGapl,1
...,...,...
135,wave_V_delayedg,1
136,waveform_ItoV_prolongedl,1
137,waveform_ItoV_prolongedg,1
138,indentifierl,1


In [22]:
edges = pd.read_csv("data/edge.csv")

# generate map from iso_code to ids of form [0, ..., num_unique_iso_codes - 1]
features = set(edges["i"])
features = features.union(set(edges["j"]))
features_to_id = {code : i for (i, code) in enumerate(features)}

In [23]:
edges['i_id'] = edges['i'].map(features_to_id)
edges['j_id'] = edges['j'].map(features_to_id)
edge_index = torch.from_numpy(edges[['i_id', 'j_id']].to_numpy(np.int64)).t()

In [24]:
EDGE_FEATURES = ["mi"]
edge_attr = torch.from_numpy(edges[EDGE_FEATURES].to_numpy(np.float32)) #extract the features from the dataset.
edge_attr = (edge_attr - edge_attr.mean(axis=0)) / (edge_attr.std(axis=0))

In [25]:
edge_index

tensor([[126,  68, 126,  ...,   5, 103,  60],
        [ 68, 126,  11,  ..., 114,  60, 103]])

In [26]:
edge_attr

tensor([[-0.4508],
        [-0.4508],
        [-0.4508],
        ...,
        [ 2.4327],
        [ 2.4327],
        [ 2.4327]])

In [27]:
# load in target values
y_df = pd.read_csv("data/out.csv ")
y_df['id'] = y_df['f'].map(features_to_id)

In [28]:
y_list = y_df.sort_values('id')["mi"].to_numpy(np.float32)
len(y_list)

140

In [29]:
y = torch.from_numpy(y_list).unsqueeze(1)

In [30]:
# load in input features
x_df = pd.read_csv("data/node.csv")
x_df['id'] = x_df['f'].map(features_to_id)

In [31]:
x_df

,Unnamed: 0,f,mi,id
0,0,age_gt_60l,1,62
1,1,age_gt_60g,1,126
2,2,airl,1,2
3,3,airg,1,68
4,4,airBoneGapl,1,84
...,...,...,...,...
135,135,wave_V_delayedg,1,24
136,136,waveform_ItoV_prolongedl,1,114
137,137,waveform_ItoV_prolongedg,1,5
138,138,indentifierl,1,103


In [32]:
x_df.sort_values("id")

,Unnamed: 0,f,mi,id
47,47,m_m_gt_2kg,1,0
104,104,mod_sn_gt_2kl,1,1
2,2,airl,1,2
32,32,history_ringingl,1,3
36,36,history_vomitingl,1,4
...,...,...,...,...
86,86,m_sn_lt_2kl,1,135
106,106,mod_sn_gt_3kl,1,136
42,42,m_cond_lt_1kl,1,137
123,123,s_sn_gt_2kg,1,138


In [33]:
x_list = x_df.sort_values("id").loc[:,"id"].to_numpy(np.int64)
x = torch.from_numpy(x_list)

In [34]:
x

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139])

In [35]:
dataset = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

In [36]:
NUM_EDGE_FEATURES = 1
EMB_SIZE = 16
NUM_EMB = x.size()[0] + 1

In [37]:
n_features = len(x)

In [38]:
# splitting the data into train, validation and test
X_train, X_test, y_train, y_test = train_test_split(pd.Series(x_list), 
                                                    pd.Series(y_list),
                                                    test_size=0.30, 
                                                    random_state=42)

In [39]:
# create train and test masks for data
train_mask = torch.zeros(n_features, dtype=torch.bool)
test_mask = torch.zeros(n_features, dtype=torch.bool)
train_mask[X_train.index] = True
test_mask[X_test.index] = True
dataset.train_mask= train_mask
dataset.test_mask = test_mask

In [40]:
NUM_VAL = len(X_test)
NUM_VAL

42

In [41]:
NUM_TRAIN = len(X_train)
NUM_TRAIN

98

In [68]:
# Basically the same as the baseline except we pass edge features 
class FEModel(torch.nn.Module):
    def __init__(self, num_features=EMB_SIZE, hidden_size=32, target_size=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_features = num_features
        self.target_size = target_size
#         self.convs = [GATConv(self.num_features, self.hidden_size, edge_dim = NUM_EDGE_FEATURES),
#                       GATConv(self.hidden_size, self.hidden_size, edge_dim = NUM_EDGE_FEATURES)]
        self.item_embedding = torch.nn.Embedding(num_embeddings=NUM_EMB, embedding_dim=EMB_SIZE)
        self.conv1 = GATConv(self.num_features, self.hidden_size, edge_dim = NUM_EDGE_FEATURES)
        self.conv2 = GATConv(self.num_features, self.hidden_size, edge_dim = NUM_EDGE_FEATURES)
        self.linear = nn.Linear(self.hidden_size, self.target_size)
       

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = torch.tensor(x)
        x = self.item_embedding(x)
        x = x.squeeze(1)   
        #print(edge_index)
        for conv in self.convs[:-1]:
            x = conv(x, edge_index, edge_attr=edge_attr) # adding edge features here!
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
        x = self.convs[-1](x, edge_index, edge_attr=edge_attr) # edge features here as well
        x = self.linear(x)
        return x

In [43]:
hyperparams = {
    'batch_size' : 1, 
    'save_loss_interval' : 5, 
    'print_interval' : 20,
    'save_model_interval' : 20,
    'n_epochs' : 200,
    'learning_rate' : 0.01
}

In [44]:
learning_rate = hyperparams['learning_rate']
batch_size = hyperparams['batch_size']
n_epochs = hyperparams['n_epochs']
save_loss_interval = hyperparams['save_loss_interval']
print_interval = hyperparams['print_interval']
save_model_interval = hyperparams['save_model_interval']

In [69]:
model = FEModel()

In [70]:
print(model)

FEModel(
  (item_embedding): Embedding(141, 16)
  (conv1): GATConv(16, 32, heads=1)
  (conv2): GATConv(16, 32, heads=1)
  (linear): Linear(in_features=32, out_features=1, bias=True)
)


In [46]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
losses = []

In [54]:
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import to_networkx

# Create batches with neighbor sampling
train_loader = NeighborLoader(
    dataset,
    num_neighbors=[5, 10],
    batch_size=2,
    input_nodes=dataset.train_mask,
)

In [55]:
 for epoch in range(n_epochs):
        epoch_loss_train = 0
        epoch_loss_val = 0
        model.train()
        for data in train_loader:
            #print(data)
            optimizer.zero_grad()
            out = model(data)
            loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
            epoch_loss_train += loss.item() 
            loss.backward()
            optimizer.step()
        
            val_loss = F.mse_loss(out[data.test_mask], data.y[data.test_mask])
            epoch_loss_val += val_loss.item()
            
        
        if epoch % print_interval == 0:
            print("Epoch: {} Train loss: {:.2e} Validation loss: {:.2e}".format(epoch, epoch_loss_train/NUM_TRAIN, epoch_loss_val/NUM_VAL))
#         losses.append((epoch, train_loss, val_loss))
#         if epoch % save_model_interval == 0:
#             # save predictions for plotting
#             model.eval()

C:\Users\mouin\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


Epoch: 0 Train loss: 1.96e-03 Validation loss: 6.84e-02
Epoch: 20 Train loss: 1.99e-03 Validation loss: 3.16e-02
Epoch: 40 Train loss: 2.07e-03 Validation loss: 2.26e-02
Epoch: 60 Train loss: 1.90e-03 Validation loss: 1.96e-02
Epoch: 80 Train loss: 2.12e-03 Validation loss: 1.72e-02
Epoch: 100 Train loss: 2.15e-03 Validation loss: 1.77e-02
Epoch: 120 Train loss: 2.16e-03 Validation loss: 1.87e-02
Epoch: 140 Train loss: 1.83e-03 Validation loss: 1.60e-02
Epoch: 160 Train loss: 2.17e-03 Validation loss: 1.69e-02
Epoch: 180 Train loss: 2.39e-03 Validation loss: 1.62e-02
